In [ ]:
"""
---- G O A L ----
1)  SIMULATE THE MOST ACURATE BALLOT DISTRIBUTION
2)  FIND POSSIBLE PATHS TO VICTORY FOR 2ND AND 3RD SEED CANDIDATES
3)  FIGURE OUT WHICH SIMULATION VARIABLES PRODUCED THE MOST ACCURATE BALLOTS 
---- P L A N ----
1)  create candidates
2)  create data structure for each candidate
        - contains dictionary: likely hood to rank each other candidate
---- SIM BEGINS ----
LOOP 
{
3)  create ballots from initial ranked choice voting
        - add to ballot -> if likely hood of selected a candidate is above X% add candidate to ballot
4)  run instant runoff and log winner
}
---- SIM STATS ----
5) Add and change parameters to simulate real life differences in ballots (threshold for additional ranking, confidence ranges of probabilites)
---- SOURCE DATA ----
6)  Find better way to get likely hood percentages (look at heat maps of leads)
7)  Consider implementing ranked choice voting strategies
---- RETRO ANALYSIS ----
8) COMPARE WHICH SIMULATIONS HAD THE CLOSEST BALLOTS TO ACTUAL DATA - LEARN WHICH FACTORS OF THE SIMULATION WERE MOST IMPORTANT


JUNE 23/24 -   MAKE DATA STRUCTURE AND SIM LOOP AND AGREGATE STATS AND GRAPHS
JUNE 23/24 -   ADD PARAMETERS THAT MAKE SIM MORE REALISTIC 
JUNE 28 -   SET UP RETROACTIVE RESEARCH EXCEPT FOR FINAL BALLOT DATA
                - CREATE COST FUNCTION BETWEEN MY BALLOTS AND TRUE BALLOTS
JUNE 29 -   DO RETROACTIVE RESEARCH - IMPORT AND FORMAT ACTUAL BALLOT DATA AND RUN
"""


In [8]:
import pyrankvote
from pyrankvote import Candidate, Ballot
import random
MAX_RANKS = 5
MIN_FAVOR = .5
ROUNDS = 30

adams = Candidate("adams")          #   0
wiley = Candidate("wiley")          #   1
garcia = Candidate("garcia")        #   2
yang = Candidate("yang")            #   3
stringer = Candidate("stringer")    #   4
morales = Candidate("morales")      #   5

candidates = [adams, wiley, garcia, yang, stringer, morales]
can_str = {"adams":adams,"wiley":wiley,"garcia":garcia,"yang":yang,"stringer":stringer,"morales":morales}
winners = {"adams": 0,"wiley":0,"garcia":0,"yang":0,"stringer":0,"morales":0}

first_votes = {}
first_votes["adams"] = 253234
first_votes["wiley"] = 177722
first_votes["garcia"] = 155812
first_votes["yang"] = 93291
first_votes["stringer"] = 40244
first_votes["morales"] = 22221

votes = 0
for values in first_votes.values():
    votes += values

for x in range(0,ROUNDS):
    favor_web = {}
    for can_a in candidates:
        favor_web[can_a.name] = {}
        for can_b in candidates:
            favor_web[can_a.name][can_b.name] = round(random.random(),2)
            if can_a.name == can_b.name:
                favor_web[can_a.name][can_b.name] = 1
                
    num_candidates = len(candidates)

    print(favor_web)

    # ADD ALL FAVOR WEB PROBABILITIES HERE

    ## yang and garcia campained together
    favor_web["yang"]["garcia"] = round(random.uniform(.5,.99),2)
    favor_web["garcia"]["yang"] = round(random.uniform(.5,.99),2)




    #BUILD BALLOTS with first choice filled
    ballots = [] 
    # ballot[x] = x individual ballots --> list of tuples 
    # ballot[x][y] = y tuples of candidate plus favorability
    # ballot[x][y][z] = z:0 is candidate obj and z:1 is favorability
    for i in range(0,votes):
        if(i>=0 and i<253234 ):
            ballots.append([(adams,1)])
        elif(i>=253234 and i<430956):
            ballots.append([(wiley,1)])
        elif(i>=430956 and i<586768):
            ballots.append([(garcia,1)])
        elif(i>=586768 and i<680059):
            ballots.append([(yang,1)])
        elif(i>=680059 and i<720303):
            ballots.append([(stringer,1)])
        else:
            ballots.append([(morales,1)])


    for ballot in ballots:
        name = ballot[0][0].name
        for other_peep in candidates:
            if name!=other_peep.name:
                if(favor_web[name][other_peep.name]>=MIN_FAVOR):
                    ballot.append((other_peep,favor_web[name][other_peep.name]))

    #FORMAT BALLOTS
    #sort by favorite
    for ballot in ballots:
        ballot.sort(key=lambda tup: tup[1], reverse=True)
        ballot = ballot[:MAX_RANKS]
    #create final data block
    ballots_formatted = []
    for ballot in ballots:
        ballots_formatted.append([a_tuple[0] for a_tuple in ballot])
    final_ballots = []
    for ballot in ballots_formatted:
        final_ballots.append(Ballot(ranked_candidates=ballot))  



    # You can use your own Candidate and Ballot objects as long as they implement the same properties and methods
    election_result = pyrankvote.instant_runoff_voting(candidates, final_ballots)

    winner = election_result.get_winners()[0]
    # Returns: [<Candidate('Al Gore (Democratic)')>]

    print(election_result)  
    winners[winner.name] += 1


, 'morales': 0.92}, 'wiley': {'adams': 0.4, 'wiley': 1, 'garcia': 0.23, 'yang': 0.23, 'stringer': 0.05, 'morales': 0.38}, 'garcia': {'adams': 0.35, 'wiley': 0.26, 'garcia': 1, 'yang': 0.91, 'stringer': 0.96, 'morales': 0.14}, 'yang': {'adams': 0.9, 'wiley': 0.97, 'garcia': 0.48, 'yang': 1, 'stringer': 0.38, 'morales': 0.03}, 'stringer': {'adams': 0.4, 'wiley': 0.65, 'garcia': 0.67, 'yang': 0.14, 'stringer': 1, 'morales': 0.15}, 'morales': {'adams': 0.66, 'wiley': 0.92, 'garcia': 0.38, 'yang': 0.42, 'stringer': 0.52, 'morales': 1}}
ROUND 1
Candidate      Votes  Status
-----------  -------  --------
adams         253234  Hopeful
wiley         177722  Hopeful
garcia        155812  Hopeful
yang           93291  Rejected
stringer       40244  Rejected
morales        22221  Rejected

ROUND 2
Candidate      Votes  Status
-----------  -------  --------
wiley         293234  Hopeful
adams         253234  Hopeful
garcia        196056  Rejected
yang               0  Rejected
stringer           0 

In [18]:
favor_web["yang"]["garcia"] = round(random.uniform(.5,.99),2)
print(favor_web["yang"]["garcia"])

0.97


In [9]:
print(winners)

{'adams': 20, 'wiley': 6, 'garcia': 4, 'yang': 0, 'stringer': 0, 'morales': 0}


In [ ]:
{'adams': 9, 'wiley': 0, 'garcia': 1, 'yang': 0, 'stringer': 0, 'morales': 0}
{'adams': 28, 'wiley': 14, 'garcia': 8, 'yang': 0, 'stringer': 0, 'morales': 0}




In [ ]:
{'adams': 
    {'adams': 1, 'wiley': 0.74, 'garcia': 0.36, 'yang': 0.24, 'stringer': 0.12, 'morales': 0.18}, 
'wiley': 
    {'adams': 0.58, 'wiley': 1, 'garcia': 0.67, 'yang': 0.97, 'stringer': 0.11, 'morales': 0.07}, 
'garcia': 
    {'adams': 0.82, 'wiley': 0.64, 'garcia': 1, 'yang': 0.87, 'stringer': 0.62, 'morales': 0.16}, 
'yang': 
    {'adams': 0.15, 'wiley': 0.56, 'garcia': 0.9, 'yang': 1, 'stringer': 0.16, 'morales': 0.44}, 
'stringer': 
    {'adams': 0.4, 'wiley': 0.89, 'garcia': 0.27, 'yang': 0.77, 'stringer': 1, 'morales': 0.83}, 
'morales': 
    {'adams': 0.58, 'wiley': 0.4, 'garcia': 0.71, 'yang': 0.07, 'stringer': 0.96, 'morales': 1}}